In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
from lib.reproduction import spectral_ranges, major_oxides
from lib.norms import Norm3Scaler, Norm1Scaler

In [52]:
from lib.full_flow_dataloader import load_full_flow_data

train, test = load_full_flow_data()

In [53]:
t3 = train.copy()

In [54]:
n_c = 2048

c1 = t3.iloc[:, :n_c]

c1.sum(axis=1).sum()
c1_n = c1.div(c1.sum(axis=1).sum(), axis=0)

In [55]:
c1_n.sum()

240.811      0.0
240.86501    0.0
240.918      0.0
240.972      0.0
241.02699    0.0
            ... 
340.621      0.0
340.66501    0.0
340.70901    0.0
340.75299    0.0
340.797      0.0
Length: 2048, dtype: float64

In [57]:
# test if the scaler is working
import numpy as np

drop_cols = major_oxides + ['Sample Name', 'ID']

# assert sum of each of the three ranges is 3 using spectrometer_wavelength_ranges
def test_norm3_scaler(test_df):
    # Simulated Data
    df = test_df.copy()

    # Initialize and fit-transform the scaler
    scaler = Norm3Scaler()
    scaler.fit(df)
    transformed_df = scaler.transform(df)

    numerical_df = transformed_df.drop(drop_cols, axis=1)

    # Verify the sum of each range is approximately 1
    for key, (start, end) in spectral_ranges.items():
        selected_columns = [col for col in numerical_df.columns if start <= float(col) <= end]
        range_sum = numerical_df[selected_columns].sum().sum()
        assert np.isclose(range_sum, 1.0), f"Sum for range {key} is not close to 1.0 but is {range_sum}"
        print(f"Sum for range {key} is close to 1.0 ({range_sum})")

# Run the test
test_norm3_scaler(train)


Sum for range UV is close to 1.0 (1.0)
Sum for range VIO is close to 1.0 (0.999993523905696)
Sum for range VNIR is close to 1.0 (0.9999999999999999)


In [58]:
def test_norm_1_scaler(test_df):
    # Simulated Data
    df = test_df.copy()

    # Initialize and fit-transform the scaler
    scaler = Norm1Scaler()
    scaler.fit(df)
    transformed_df = scaler.transform(df)

    numerical_df = transformed_df.drop(drop_cols, axis=1)

    # Verify the sum of each range is approximately 1
    selected_columns = [col for col in numerical_df.columns if float(col)]
    range_sum = numerical_df[selected_columns].sum().sum()
    assert np.isclose(range_sum, 1.0), f"Sum is not close to 1.0 but is {range_sum}"
    print(f"Sum is close to 1.0 ({range_sum})")

# Run the test
test_norm_1_scaler(train)

Sum is close to 1.0 (1.0000000000000002)
